In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import geopandas as gpd
from shapely import wkt
import matplotlib.pyplot as plt
from geodatasets import get_path
import numpy as np
import xarray as xr

import sys
sys.path.append('..')
from scripts.dicts import *

/var/folders/cv/s_h5gzxd0h71t_d875wqxgzh0000gn/T/ipykernel_98957/3273420535.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Pathdata: /Volumes/imau01/rapid/RACMO2.3p2/FGRN055


In [2]:
pathKing = '/Users/annek/Library/CloudStorage/OneDrive-SharedLibraries-NIOZ/PhD Anneke Vries - General/FWclean/data/raw/solid_discharge/King2020_doi_10_5061_dryad_qrfj6q5cb__v20200429/GrIS_D_1985-2018.xlsx'

In [28]:
ds_D_king = pd.read_excel(pathKing, header=[0,3], index_col=0, parse_dates=True)


/var/folders/cv/s_h5gzxd0h71t_d875wqxgzh0000gn/T/ipykernel_98957/1203650050.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ds_D_king = pd.read_excel(pathKing, header=[0,3], index_col=0, parse_dates=True)


In [32]:
ds_King_meta = pd.read_excel(pathKing, header=[0, 1], nrows=1, index_col=0)
ds_King_meta

Number_Name '001_79fjorden'             '002_academy'              \
coordinates      'latitude' 'longitude'    'latitude' 'longitude'   
NaN               79.501618  -23.392803     81.552332  -33.046728   

Number_Name '003_alangordliup'             '004_alangorssupsermia'  \
coordinates         'latitude' 'longitude'              'latitude'   
NaN                  68.814959  -50.269235               73.098737   

Number_Name             '005_alison'              ... '230_vestifjord'  \
coordinates 'longitude'   'latitude' 'longitude'  ...       'latitude'   
NaN          -54.966186    74.656297  -55.988739  ...        70.307794   

Number_Name             '231_walterhausen'             '232_west krista dan'  \
coordinates 'longitude'         'latitude' 'longitude'            'latitude'   
NaN          -29.577861           74.08635  -25.375273             69.965698   

Number_Name             '233_yngvarnielson'              \
coordinates 'longitude'          'latitude' 'longitude'   
NaN          -28.311832           76.357079   -64.02859   

Number_Name '234_zachariaeisstrom'              
coordinates             'latitude' 'longitude'  
NaN                      79.013814  -20.787433  

[1 rows x 468 columns]

In [25]:
ds_D_king

,Time,'001_79fjorden','001_79fjorden'.1,'002_academy','002_academy'.1,'003_alangordliup','003_alangordliup'.1,'004_alangorssupsermia','004_alangorssupsermia'.1,'005_alison',...,'230_vestifjord','230_vestifjord'.1,'231_walterhausen','231_walterhausen'.1,'232_west krista dan','232_west krista dan'.1,'233_yngvarnielson','233_yngvarnielson'.1,'234_zachariaeisstrom','234_zachariaeisstrom'.1
4,1985-01-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1985-02-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1985-03-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1985-04-15,NaN,NaN,1.168851,0.572353,1.242484,0.466054,NaN,NaN,4.476559,...,2.438552,0.783983,1.57603,0.849881,0.158769,0.063918,NaN,NaN,NaN,NaN
8,1985-05-16,11.129603,2.952281,1.097983,0.55968,1.011128,0.402917,NaN,NaN,4.601464,...,2.790726,0.867083,1.505106,0.767661,0.16234,0.076198,NaN,NaN,9.758037,2.302118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,2018-08-16,14.309279,2.461213,0.875645,0.309621,0.573949,0.13256,0.120502,0.070275,6.725226,...,2.644925,0.829511,1.060872,0.385743,0.141186,0.06581,0.778173,0.197586,17.186485,3.354786
408,2018-09-15,14.102614,2.373028,0.739661,0.319972,0.551529,0.136161,0.120502,0.070275,6.66321,...,2.644925,0.829511,1.060872,0.385743,0.141186,0.06581,0.778173,0.197586,16.240398,3.059839
409,2018-10-16,14.043295,2.349051,0.721989,0.324335,0.550892,0.136357,0.120502,0.070275,6.657329,...,2.644925,0.829511,1.060872,0.385743,0.141186,0.06581,0.778173,0.197586,16.014766,2.91217
410,2018-11-15,14.043295,2.349051,0.721989,0.324335,0.550892,0.136357,0.120502,0.070275,6.657329,...,2.644925,0.829511,1.060872,0.385743,0.141186,0.06581,0.778173,0.197586,16.014766,2.91217
